In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
scores_phase = pd.read_csv('../data/ec109_mismatch_old_match_old_phase_scores.csv')

In [2]:
df = pd.read_csv('../data/mismatch_old_behav_ec109.csv')

In [ ]:
def downsample_df(df, dsrate):
    df['datetime'] = pd.to_datetime(df['time'])
    

In [63]:
tmp.iloc[0:data.time.max(),:]

,events,blockno,elecs,freqs,time,pow,phase,response,view1,view2,view3,datetime
0,1,1,1,1,1,2.464962,-2.040536,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000001
35100,1,1,1,1,2,2.453640,-2.010732,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000002
70200,1,1,1,1,3,2.442230,-1.980846,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000003
105300,1,1,1,1,4,2.430764,-1.950999,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000004
140400,1,1,1,1,5,2.419358,-1.921266,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000005
175500,1,1,1,1,6,2.407965,-1.891733,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000006
210600,1,1,1,1,7,2.396432,-1.862377,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000007
245700,1,1,1,1,8,2.384698,-1.833303,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000008
280800,1,1,1,1,9,2.372925,-1.804287,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000009
315900,1,1,1,1,10,2.361011,-1.775453,2,0.509735,0.30005,0.035447,1970-01-01 00:00:00.000000010


In [60]:
newdf = pd.DataFrame()
for inds, df in data.groupby(by=['events', 'elecs', 'freqs', 'blockno']):
    tmp = df.iloc[0:751,:]
    break
    tmp = tmp.resample('25ns', on='datetime').mean()
    tmp.reset_index(drop=True, inplace=True)
    newdf = pd.concat([newdf, tmp])
newdf = newdf.round({'time':0})    
newdf.reset_index(drop=True, inplace=True)    

In [61]:
dsrate=15
tmp.resample(f'{dsrate}ns', on='datetime').mean()

,events,blockno,elecs,freqs,time,pow,phase,response,view1,view2,view3
datetime,,,,,,,,,,,
1970-01-01 00:00:00.000000000,1.0,1.0,1.0,1.0,7.5,2.389981,-1.849062,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000015,1.0,1.0,1.0,1.0,22.0,2.217858,-1.447794,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000030,1.0,1.0,1.0,1.0,37.0,2.042263,-1.078791,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000045,1.0,1.0,1.0,1.0,52.0,1.884469,-0.760976,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000060,1.0,1.0,1.0,1.0,67.0,1.766920,-0.489638,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000075,1.0,1.0,1.0,1.0,82.0,1.710398,-0.239011,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000090,1.0,1.0,1.0,1.0,97.0,1.719279,0.015471,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000105,1.0,1.0,1.0,1.0,112.0,1.786889,0.306289,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000120,1.0,1.0,1.0,1.0,127.0,1.882831,0.625122,2.0,0.509735,0.30005,0.035447


In [ ]:
dsrate=15
ds_string = f'{dsrate}ns'


In [33]:
x = pd.DataFrame(np.arange(-750,751,step=2),index=np.arange(1,752), columns=['time'])
x

,time
1,-750
2,-748
3,-746
4,-744
5,-742
6,-740
7,-738
8,-736
9,-734
10,-732


In [48]:
xx = tmp.resample('15ns', on='datetime').mean()
xx = xx.round({'time':0})

In [50]:
t_idx = np.arange(1,126)
t_idx

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125])

In [54]:
xx.loc[(xx['time'].isin(t_idx))]

,events,blockno,elecs,freqs,time,pow,phase,response,view1,view2,view3
datetime,,,,,,,,,,,
1970-01-01 00:00:00.000000000,1.0,1.0,1.0,1.0,8.0,2.389981,-1.849062,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000015,1.0,1.0,1.0,1.0,22.0,2.217858,-1.447794,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000030,1.0,1.0,1.0,1.0,37.0,2.042263,-1.078791,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000045,1.0,1.0,1.0,1.0,52.0,1.884469,-0.760976,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000060,1.0,1.0,1.0,1.0,67.0,1.766920,-0.489638,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000075,1.0,1.0,1.0,1.0,82.0,1.710398,-0.239011,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000090,1.0,1.0,1.0,1.0,97.0,1.719279,0.015471,2.0,0.509735,0.30005,0.035447
1970-01-01 00:00:00.000000105,1.0,1.0,1.0,1.0,112.0,1.786889,0.306289,2.0,0.509735,0.30005,0.035447


In [ ]:
sns.barplot(x='starttime', y='auc_z', hue='startfreq', data=scores)

In [ ]:
scores['auc_z'] = scores['auc_z'].str.split(expand=True)[1].astype(dtype='float64')

In [ ]:
scores.auc_z.dtype

In [ ]:
r1 = data[data['response']==1]

In [ ]:
r1['class']=1
r1['blockno']=1

In [ ]:
keep_pow=False
keep_phase=True

if keep_pow and keep_phase:
    features = 'all'
elif keep_pow and not keep_phase:
    features = 'pow'
elif not keep_pow and keep_phase:
    features= 'phase'  
features    

In [ ]:
def gen_wide_df(df, f_idx, t_idx, keep_pow, keep_phase):
    """

    :param df:
    :param f_idx:
    :param t_idx:
    :return: df
    """

    df = df.loc[(df['time'].isin(t_idx)) & (df['freqs'].isin(f_idx))]

    # z-score within block
    gb = df.groupby(['elecs', 'freqs', 'blockno'])
    if keep_pow:
        df['zpow'] = gb['pow'].apply(zscore)
    if keep_phase:
        df['sin'] = gb['phase'].apply(np.sin)
        df['cos'] = gb['phase'].apply(np.cos)

    df.drop(['pow', 'phase'], inplace=True, axis=1)

    df = pd.pivot_table(df,
                        index=['class', 'blockno', 'events'],
                        columns=['elecs', 'freqs', 'time'],
                        aggfunc=[lambda x: x])

    elecs = df.columns.get_level_values('elecs')
    freqs = df.columns.get_level_values('freqs')
    time = df.columns.get_level_values('time')
    ncol = len(elecs)
    df.reset_index(drop=False, inplace=True)
    return df, elecs, freqs, time, ncol

In [ ]:
df, elecs, freqs, time, ncol = gen_wide_df(r1, list(range(1,14)), list(range(1,125)), keep_pow, keep_phase)

In [ ]:
view_conds = [f'view{x}' for x in range(1,4)]
cond='view1'

In [ ]:
view_conds.append('response')
view_conds.remove(cond)

In [ ]:
view_conds

In [ ]:
data.columns

In [ ]:
cols_to_drop = [f'view{x}' for x in range(1,4)]

In [ ]:
cols_to_drop.append('response')

In [ ]:
cols_to_drop.remove(cond)

In [ ]:
cols_to_drop

In [ ]:
cols_to_drop = [f'view{x}' for x in range(1,4)]+['response']

In [ ]:
cols_to_drop

In [ ]:
np.array([10,50,100])